# Definition der Ziele und Daten
Im Bereich der Personenfahrzeuge wird die Entwicklung des Fahrzeugbestands in der Schweiz untersucht. Ziel ist es, auf Basis öffentlich verfügbarer Daten Erkenntnisse über regionale Unterschiede sowie mögliche Zusammenhänge zwischen relevanten Einflussfaktoren zu gewinnen. Im Fokus stehen hierbei die Elektrofahrzeuge und deren infrastrukturelle sowie wirtschaftliche Rahmenbedingungen. Die folgenden Forschungsfragen werden im Detail analysiert.

## Entwicklung der Personenfahrzeuge in der Schweiz .
Wie hat sich der Bestand an Personenfahrzeugen in der Schweiz in den Jahren zwischen 2013-2023 Jahren entwickelt?

## Vergleich von den einzelnen Kantonen zum nationalen Wachstum
Wie unterscheidet sich die Entwicklung der Fahrzeugbestände in den einzelnen Kantonen im Vergleich zur gesamtschweizerischen Entwicklung?

## Vergleich der drei Haupttreibstofftypen
Wie unterscheiden sich die Entwicklungen von Fahrzeugen mit Benzin-, Diesel- und Elektroantrieb?

## Zusammenhang zwischen dem Wachstum von Elektrofahrzeugen und öffentlichen Ladestationen
Besteht ein Zusammenhang zwischen dem Zuwachs an Elektrofahrzeugen und dem Ausbau der öffentlichen Ladeinfrastruktur in der Schweiz?

## Zusammenhang zwischen Strompreisen, öffentlichen Ladestationen und Elektrofahrzeugen
Gibt es einen  Zusammenhang zwischen den kantonalen Strompreisen, der Verfügbarkeit öffentlicher Ladestationen und der Anzahl an Elektrofahrzeugen?

# Datenquellen

Für die Analyse der Entwicklung und Trends im Bereich Personenfahrzeuge wurden unterschiedliche Quellen verwendet. Alle Daten konnten als CSV-Datei bezogen werden. Die Rohdaten sind im GitHub Repository abgelegt.

*   Bestand der Strassenfahrzeuge nach Gemeinde ab dem Jahr 2010 | Bundesamt für Statistik | Nutzungsbedingungen:OPEN-BY-ASK| Veröffentlicht: 30.01.2025 | Format:csv | Quelle: https://www.bfs.admin.ch/bfs/de/home/statistiken/mobilitaet-verkehr/verkehrsinfrastruktur-fahrzeuge/fahrzeuge/strassenfahrzeuge-bestand-motorisierungsgrad.assetdetail.33827764.html

*   Kennzahlen öffentliche Ladeinfrastruktur Elektromobilität | Bundesamt für Energie BFE |Nutzungsbedingungen:OPEN-USE | Veröffentlicht: 02.04.2025| Format: csv | Quelle: https://www.uvek-gis.admin.ch/BFE/ogd/57/ich_tanke_strom_Kennzahlen_monatlich.csv



*   Amtliches Ortschaftenverzeichnis | Bundesamt für Landestopografie swisstopo| Nutzungsbedingungen:OPEN-USE | Veröffentlicht: 10.10.2024| Format: csv | https://www.swisstopo.admin.ch/de/amtliches-ortschaftenverzeichnis


*   Median Strompreis für die Schweiz |Schweizer Eidgenossenschaft |Nutzungsbedingungen:OPEN-USE | Veröffentlicht: 01.01.2021| Format: csv |https://register.ld.admin.ch/opendataswiss/org/elcom











# Laden und Importieren der Libaries
Die Verarbeitung und Darstellung der Daten erfolgt mit den untenstehenden Libarys, welche in einem ersten Schritt importiert werden.

In [ ]:
#Load necessary packages/libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.stats import linregress
from plotly.subplots import make_subplots
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.ticker as mticker
import pandas as pd
!pip install dash
!pip install mplcursors


# Laden der einzelnen Datensets
Die Datensets wurden mit der Programmiersprache Python unter Verwendung der Bibliothek Pandas eingelesen. Alle Dateien lagen im CSV-Format vor, wobei je nach Quelle unterschiedliche Trennzeichen und Zeichencodierungen berücksichtigt werden mussten.

In [ ]:
# Daten einlesen
df_car = pd.read_csv("/content/sample_data/Anzahl Fahrzeuge pro Typ.csv",
                 sep=';',
                 encoding='latin-1',
                 quotechar='"')


df_city = pd.read_csv("/content/sample_data/Ortschaften.csv",
                 sep=';',
                 encoding='utf-8',
                 quotechar='"')

df_strom = pd.read_csv("/content/sample_data/Median Strompreis per Kanton-full.csv", delimiter=",")

df_lade = pd.read_csv("/content/sample_data/ladestationen.csv")


Um zu überprüfen, ob die Daten korrekt improtiert wurden, wird von allen Datensets die ersten fünf Einträge abgerufen.

In [ ]:
df_car.head()

,Gemeinde,Fahrzeuggruppe,Treibstoff,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,1 Aeugst am Albis,Personenwagen,Benzin,863,881,891,881,856,845,822,815,816,809,804,789,778,774,724
1,1 Aeugst am Albis,Personenwagen,Diesel,175,204,229,250,276,288,306,316,318,326,329,321,311,292,287
2,1 Aeugst am Albis,Personenwagen,Benzin-elektrisch: Normal-Hybrid,3,5,7,11,13,13,18,16,20,22,30,43,49,57,74
3,1 Aeugst am Albis,Personenwagen,Benzin-elektrisch: Plug-in-Hybrid,0,0,0,0,0,0,1,2,7,7,12,20,23,26,29
4,1 Aeugst am Albis,Personenwagen,Diesel-elektrisch: Normal-Hybrid,0,0,1,1,0,0,0,2,2,3,2,5,5,7,4


In [ ]:
df_city.head()

,Ortschaftsname,PLZ,Zusatzziffer,Gemeindename,BFS-Nr,Kantonskürzel,E,N,Sprache,Validity
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,2679402.872,1235842.010,de,2008-07-01
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,2679815.372,1237404.310,de,2008-07-01
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,2675280.133,1238108.286,de,2008-07-01
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,2676852.012,1236929.718,de,2008-07-01
4,Bonstetten,8906,0,Bonstetten,3,ZH,2677412.150,1241078.278,de,2008-07-01


In [ ]:
df_strom.head()

,Kanton,Verbrauchsprofile typischer Haushalte,Periode,Produkt,Total exkl. MWST (Rp./kWH),Netznutzung (Rp./kWH),Energielieferungkosten (Rp./kWH),Abgaben an das Gemeinwesen (Rp./kWH),Förderabgaben (KEV) (Rp./kWH)
0,St. Gallen,H4,2018,Günstigstes Produkt,18.400000,8.703333,6.138000,0.60,2.3
1,Wallis,C3,2022,Günstigstes Produkt,17.489167,6.480000,7.929167,0.78,2.3
2,Tessin,H5,2019,Günstigstes Produkt,17.292400,6.682933,6.139467,2.17,2.3
3,Schwyz,H4,2023,Günstigstes Produkt,24.661556,10.672619,11.231000,0.85,2.3
4,Zug,H4,2019,Günstigstes Produkt,17.648952,8.615619,6.733333,0.00,2.3


In [ ]:
df_lade.head()

,year,month,chargingPower_10kW_count,chargingPower_21kW_count,chargingPower_42kW_count,chargingPower_100kW_count,chargingPower_100pluskW_count,chargingPower_AG_count,chargingPower_AG_sum,chargingPower_AI_count,...,stations_SH_count,stations_SO_count,stations_SZ_count,stations_TG_count,stations_TI_count,stations_UR_count,stations_VD_count,stations_VS_count,stations_ZG_count,stations_ZH_count
0,2020,11,2175,498,3210,661,563,326,11660,13,...,52,196,86,109,371,32,484,332,122,620
1,2020,12,2291,516,3353,671,564,333,11722,13,...,53,202,85,113,396,32,513,348,124,634
2,2021,1,2354,529,3483,719,584,353,13209,13,...,54,209,90,113,426,32,537,370,141,669
3,2021,2,2382,538,3558,746,609,392,16806,13,...,60,212,100,111,431,32,549,382,174,698
4,2021,3,2414,547,3632,748,614,400,17190,13,...,58,216,102,108,435,32,554,391,172,721


# Formatieren und Bereinigen der Daten
Da nicht alle Informationen aus den unterschiedlichen Datenquellen für die Analyse benötigt werden, werden in diesem Schritt die einzelnen Datensets bereinigt. Beispielsweise wird aus der df_car die führende Zahl aus der Gemeindespalte entfernt.

In [ ]:
#Formatieren der Daten
##df_car
# Entferne führende Zahl aus der Gemeinde-Spalte
df_car = df_car.drop(columns=['2010', '2011', '2012'])
df_car['Gemeinde'] = df_car['Gemeinde'].str.replace(r'^\d+\s*', '', regex=True)


##df_City
# Spalten 'Validity' und 'Sprache' entfernen
df_city = df_city.drop(columns=['Validity', 'Sprache'])


##df_strom
##Kantonsmapping durchführen
kanton_mapping = {
    "Aargau": "AG", "Appenzell Ausserrhoden": "AR", "Appenzell Innerrhoden": "AI",
    "Basel-Landschaft": "BL", "Basel-Stadt": "BS", "Bern": "BE", "Freiburg": "FR",
    "Genf": "GE", "Glarus": "GL", "Graubünden": "GR", "Jura": "JU", "Luzern": "LU",
    "Neuenburg": "NE", "Nidwalden": "NW", "Obwalden": "OW", "Schaffhausen": "SH",
    "Schwyz": "SZ", "Solothurn": "SO", "St. Gallen": "SG", "Tessin": "TI",
    "Thurgau": "TG", "Uri": "UR", "Waadt": "VD", "Wallis": "VS", "Zug": "ZG",
    "Zürich": "ZH"
}
df_strom['Kantonskürzel'] = df_strom['Kanton'].map(kanton_mapping)


##df_lade
## Entfernen der Spalten
df_lade = df_lade.drop(columns=[
    'chargingPower_10kW_count',
    'chargingPower_21kW_count',
    'chargingPower_42kW_count',
    'chargingPower_100kW_count',
    'chargingPower_100pluskW_count'
], errors='ignore')

# Mergen der beiden Datensätze df_car und df_city
Um den Fahrzeugbestand auf Gemeindeebene mit zusätzlichen geografischen Informationen anzureichern, wurde ein Datenabgleich zwischen zwei Datensätzen durchgeführt. Konkret wurden die Fahrzeugdaten `df_car` mit einem Gemeindedatensatz `df_city` zusammengeführt. Die Verbindung erfolgte über den Gemeindename: Die Spalte Gemeinde aus dem Fahrzeugdatensatz wurde mit der Spalte Gemeindename aus dem Gemeindedatensatz abgeglichen. Dabei wurde ein Left Join verwendet, um sicherzustellen, dass alle Gemeinden aus dem Fahrzeugdatensatz erhalten bleiben.

Nach dem Zusammenführen enthielt der neue Datensatz `df_car_city` neben den ursprünglichen Informationen auch die Postleitzahl PLZ sowie das zugehörige Kantonskürzel. Der erweiterte Datensatz wird gespeichert als CSV-Datei unter dem Namen `output_mit_plz.csv`.

In [ ]:
# Mergen der beiden DataFrames basierend auf der Spalte "Ort"
df_car_city = pd.merge(df_car, df_city[['Gemeindename', 'PLZ', 'Kantonskürzel']], left_on='Gemeinde', right_on='Gemeindename', how='left')
# Speichern der neuen CSV-Datei mit der hinzugefügten PLZ
df_car_city.to_csv('output_mit_plz.csv', index=False, encoding='utf-8', sep=';', quotechar='"')

# Daten analysieren
Nachfolgend werden die erfassten Daten in Hinblick auf die Forschungsfragen analysiert und beschrieben. Die Daten basieren auf den aufbereiteten Daten aus den vorhergehenden Kapiteln.

## Wachstum der Anzahl Fahrzeuge in der Schweiz.
In der Schweiz ist der Bestand der Fahrzeuge zwischen den Jahren 2013 und 2023 um 1'484'084 Personenfahrzeuge gestiegen. Die prozentuale Zunahme entspricht somit 5.9 %, dies entspricht einer durchschnittlichen jährliche Wachstumsrate von 0.6 %.

In [ ]:
# Jahres-Spalten definieren (moving this up to fix the reference issue)
jahr_spalten = [str(year) for year in range(2013, 2024)]

# Daten vorbereiten
df_car_city[jahr_spalten] = df_car_city[jahr_spalten].apply(pd.to_numeric, errors='coerce')
df_car_city_summe = df_car_city[jahr_spalten].sum(axis=0)
df_car_city_summe.index = df_car_city_summe.index.astype(int)

# Wachstum zwischen 2013 und 2023 berechnen
wachstum_gesamt = df_car_city_summe[2023] - df_car_city_summe[2013]
wachstum_prozent = (df_car_city_summe[2023] / df_car_city_summe[2013] - 1) * 100

# Jährliche durchschnittliche Wachstumsrate (CAGR) berechnen
jahre = 2023 - 2013
cagr = (pow(df_car_city_summe[2023] / df_car_city_summe[2013], 1/jahre) - 1) * 100

# Gesamtsumme der Fahrzeuge für die ganze Schweiz pro Jahr berechnen
df_schweiz = df_car_city[jahr_spalten].sum()
df_schweiz = pd.DataFrame({'Jahr': df_schweiz.index.astype(int), 'Anzahl': df_schweiz.values})

# IBCS Farbschema
IBCS_BLACK = '#000000'  # Primärfarbe für Ist-Werte nach IBCS
IBCS_GRAY = '#888888'   # Für weniger wichtige Elemente

# Plot erstellen (IBCS empfiehlt flachere Darstellung)
fig = go.Figure()

# Minimalwert für Y-Achse
min_y = min(df_schweiz['Anzahl'])
max_y = max(df_schweiz['Anzahl'])

# Werte formatieren
def format_in_millions(value):
    """Formatiert Werte in Millionen mit einer Dezimalstelle"""
    return f"{round(value/1000000, 1)}M"

# Vollständige Werte für Hover-Info formatieren
hover_values = [f"{int(val):,}".replace(",", "'") for val in df_schweiz['Anzahl']]
# Kompakte Werte für die Anzeige
text_values = [format_in_millions(val) for val in df_schweiz['Anzahl']]

# Trace erstellen
fig.add_trace(go.Scatter(
    x=df_schweiz['Jahr'],
    y=df_schweiz['Anzahl'],
    mode='lines+markers+text',
    name='Schweiz Gesamt',
    line=dict(color=IBCS_BLACK, width=1.5),
    marker=dict(size=6, color=IBCS_BLACK),
    text=text_values,
    textposition='top center',
    textfont=dict(size=9, color=IBCS_BLACK),
    hovertemplate='<b>%{x}</b><br>Anzahl Fahrzeuge: %{customdata}<extra></extra>',
    customdata=hover_values
))

# Annotation für den Minimalwert
min_annotation = dict(
    x=df_schweiz['Jahr'].min() - 0.5,
    y=min_y,
    xref='x',
    yref='y',
    text=format_in_millions(min_y),
    showarrow=False,
    font=dict(color=IBCS_GRAY, size=9),
    align='right',
    xanchor='right'
)

# Quellen-Annotation
source_annotation = dict(
    x=0.01,
    y=-0.15,
    xref='paper',
    yref='paper',
    text='Quelle: Gesamtanzahl registrierter Fahrzeuge in der Schweiz, 2013-2024',
    showarrow=False,
    font=dict(size=8, color=IBCS_GRAY),
    align='left',
    xanchor='left'
)

# Information zu den Millionen-Werten
million_note = dict(
    x=0.99,
    y=-0.15,
    xref='paper',
    yref='paper',
    text='Angaben in Millionen (M)',
    showarrow=False,
    font=dict(size=8, color=IBCS_GRAY),
    align='right',
    xanchor='right'
)

# Layout anpassen gemäss IBCS
fig.update_layout(
    # Titel gemäss IBCS (links, zweistufig)
    title={
        'text': f"<b>Fahrzeugbestand in der Schweiz</b><br>"
                f"<span style='font-size:10px;color:#888888'>"
                f"Entwicklung 2013–2023 – Absolute Zunahme: {wachstum_gesamt:,}".replace(",", "'") + " "
                f"({wachstum_prozent:.1f}%) • Durchschnittliches Wachstum (CAGR): {cagr:.1f}%"
                f"</span>",
        'x': 0.01,
        'xanchor': 'left',
        'font': {'size': 14, 'color': IBCS_BLACK}
    },
    # Achsenformatierung gemäss IBCS
    xaxis={
        'showgrid': False,  # Kein vertikales Gitter nach IBCS
        'showline': True,
        'linecolor': IBCS_GRAY,
        'linewidth': 0.5,
        'ticks': 'outside',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'title': {'text': 'Jahr', 'font': {'size': 10, 'color': IBCS_BLACK}},
        'tickmode': 'array',
        'tickvals': df_schweiz['Jahr'],
        'range': [df_schweiz['Jahr'].min() - 0.5, df_schweiz['Jahr'].max() + 0.5]
    },
    yaxis={
        'showgrid': False,  # Keine Gitterlinien für IBCS-Konformität
        'showline': False,  # Keine Y-Achsenlinie für IBCS-Konformität
        'showticklabels': False,  # Keine Y-Achsenbeschriftungen gemäss IBCS
        'range': [min_y * 0.95, max_y * 1.05],  # Bereich mit leichtem Puffer
        'zeroline': False
    },
    # Sonstige Formatierung
    plot_bgcolor='white',    # Weisser Hintergrund nach IBCS
    paper_bgcolor='white',
    margin={'t': 100, 'b': 50, 'l': 50, 'r': 30},
    height=400,  # Flacheres Format nach IBCS
    width=800,
    showlegend=False,  # Keine Legende nötig
    annotations=[min_annotation, source_annotation, million_note]  # Alle Annotationen
)

# Wachstum berechnen
wachstum_absolut = df_schweiz['Anzahl'].iloc[-1] - df_schweiz['Anzahl'].iloc[0]
wachstum_prozent = (df_schweiz['Anzahl'].iloc[-1] / df_schweiz['Anzahl'].iloc[0] - 1) * 100
jahre_anzahl = len(df_schweiz) - 1
cagr = (pow(df_schweiz['Anzahl'].iloc[-1] / df_schweiz['Anzahl'].iloc[0], 1/jahre_anzahl) - 1) * 100

# Wachstumsinfo als Annotation hinzufügen
growth_annotation = dict(
    x=0.99,
    y=0.01,
    xref='paper',
    yref='paper',
    text=f"Gesamtwachstum: +{wachstum_prozent:.1f}%<br>CAGR: +{cagr:.1f}%",
    showarrow=False,
    font=dict(size=9, color=IBCS_BLACK),
    align='right',
    xanchor='right'
)
fig.add_annotation(growth_annotation)

# Anzeigen des Diagramms
fig.show()

## Entwicklung des Fahrzeugbestands in den unterschiedlichen Kantonen
In der untenstehenden Grafik wird die Entwicklung des Fahrzeugbestands in den verschiedenen Kantonen aufgezeigt im Vergleich zur prozentualen Wachstumsrate der Schweiz.

In [ ]:
# Jahres-Spalten definieren
jahre = [str(year) for year in range(2013, 2025)]

# Gruppieren nach Kanton und Summieren der Fahrzeuge pro Jahr
df_kanton_jahr = df_car_city.groupby('Kantonskürzel')[jahre].sum()
df_kanton_jahr = df_kanton_jahr.transpose()  # Jahre als Zeilen, Kantone als Spalten
df_kanton_jahr.index = df_kanton_jahr.index.astype(int)
df_kanton_jahr.index.name = 'Jahr'
df_kanton_jahr.reset_index(inplace=True)

# Prozentuale Veränderung berechnen (Jahr-zu-Jahr)
df_prozent = pd.DataFrame({'Jahr': df_kanton_jahr['Jahr']})
for kanton in df_kanton_jahr.columns[1:]:
    df_prozent[kanton] = df_kanton_jahr[kanton].pct_change() * 100

# Nationale Veränderung (CH) berechnen
df_kanton_jahr['CH'] = df_kanton_jahr.iloc[:, 1:].sum(axis=1)
df_prozent['CH'] = df_kanton_jahr['CH'].pct_change() * 100

# NaN-Werte für das erste Jahr entfernen
df_prozent = df_prozent.dropna()

# IBCS Farbschema
IBCS_BLACK = '#000000'
IBCS_GRAY = '#888888'
IBCS_LIGHT_GRAY = '#dddddd'
IBCS_GREEN = '#00cc00'
IBCS_RED = '#cc0000'

# Funktion für Prozentformatierung
def format_percent(value):
    return f"{value:.1f}%"

# Plot erstellen
fig = make_subplots(rows=1, cols=1, subplot_titles=[""])

# Nationale Spur (immer sichtbar)
text_values_ch = [format_percent(val) for val in df_prozent['CH']]
colors_ch = [IBCS_GREEN if val >= 0 else IBCS_RED for val in df_prozent['CH']]

trace_ch = go.Scatter(
    x=df_prozent['Jahr'],
    y=df_prozent['CH'],
    name='Nationale Entwicklung (CH)',  # Klarerer Name für die Legende
    mode='lines+markers+text',
    line=dict(color=IBCS_BLACK, width=2),
    marker=dict(size=6),
    text=text_values_ch,
    textposition='bottom center',  # Beschriftung nach rechts verschoben
    textfont=dict(size=9, color=IBCS_BLACK),
    hoverinfo='x+y+name',
    visible=True
)
fig.add_trace(trace_ch)

# Beschriftung "CH" mit Prozentwert am Ende der Linie hinzufügen
ch_last_value = df_prozent['CH'].iloc[-1]
ch_label = f"CH ({format_percent(ch_last_value)})"

# Bestimmen des ersten sichtbaren Kantons
first_kanton = df_prozent.columns[1]

# Spuren für jeden Kanton hinzufügen
traces = []
for kanton in df_prozent.columns[1:]:
    if kanton == 'CH':
        continue  # CH ist separat als Linie

    text_values = [format_percent(val) for val in df_prozent[kanton]]
    colors = [IBCS_GREEN if val >= 0 else IBCS_RED for val in df_prozent[kanton]]

    trace = go.Bar(
        x=df_prozent['Jahr'],
        y=df_prozent[kanton],
        name=f'Kanton {kanton}',  # Klarerer Name für die Legende
        marker=dict(color=colors),
        text=text_values,
        textposition='outside',
        textfont=dict(size=9, color=IBCS_BLACK),
        visible=(kanton == first_kanton),
        hoverinfo='x+y+name'
    )
    fig.add_trace(trace)
    traces.append(trace)

# Min/Max Y-Achse
y_min = min(0, df_prozent.iloc[:, 1:].min().min() * 1.5)
y_max = max(0, df_prozent.iloc[:, 1:].max().max() * 1.5)

# Annotationen
zero_line = dict(
    type='line',
    xref='paper',
    yref='y',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    line=dict(color=IBCS_BLACK, width=1)
)

source_annotation = dict(
    x=0.01,
    y=-0.15,
    xref='paper',
    yref='paper',
    text='Quelle: Prozentuale Veränderung registrierter Fahrzeuge nach Kanton, 2013–2024',
    showarrow=False,
    font=dict(size=8, color=IBCS_GRAY),
    align='left',
    xanchor='left'
)

# Y-Achsenbeschriftung hinzufügen
y_axis_title = dict(
    x=-0.05,
    y=0.5,
    xref='paper',
    yref='paper',
    text='Veränderung in %',
    showarrow=False,
    font=dict(size=10, color=IBCS_BLACK),
    textangle=-90
)

# Kantonsnamen kürzen
def shorten_kanton(kanton_name):
    return kanton_name[:3]

# Beschriftung "CH" am Ende der Linie
fig.add_trace(go.Scatter(
    x=[df_prozent['Jahr'].iloc[-1] + 0.2],
    y=[df_prozent['CH'].iloc[-1]],
    mode='text',
    text=['%-Entwicklung CH'],
    textposition='top right',
    showlegend=False,
    textfont=dict(size=11, color=IBCS_BLACK)
))

# Dropdown-Menü
buttons = []
for i, kanton in enumerate(df_prozent.columns[1:]):
    if kanton == 'CH':
        continue

    kanton_index = list(df_prozent.columns[1:]).index(kanton)  # z.B. 0 für AG, 1 für AI etc.
    visible = [True]  # CH-Spur immer sichtbar
    visible += [i == kanton_index for i in range(len(df_prozent.columns) - 2)]  # n-2 Kantone ohne CH und aktuelle Auswahl

    kurz_kanton = shorten_kanton(kanton)

    buttons.append(dict(
        label=kurz_kanton,
        method='update',
        args=[
            {'visible': visible},
            {
                'title': {
                    'text': f'<b>Jährliche Veränderung des Fahrzeugbestands in {kanton}</b><br>'
                            f'<span style="font-size:10px;color:#888888">Basis 2013, Prozentuale Veränderung des Fahrzeugbestands in den einzelnen Kantonen inkl. nationale Entwicklung (CH), 2013–2023</span>',
                    'x': 0.01,
                    'xanchor': 'left'
                },
                'annotations': [source_annotation, y_axis_title],
                'legend': {
                    'title': {'text': 'Legende:'},
                    'y': 1.1,
                    'x': 0.5,
                    'xanchor': 'center',
                    'orientation': 'h'
                }
            }
        ]
    ))

# Layout
fig.update_layout(
    title={
        'text': f'<b>Jährliche Veränderung des Fahrzeugbestands in {first_kanton}</b><br>'
                f'<span style="font-size:10px;color:#888888">Basis 2013, Prozentuale Veränderung des Fahrzeugbestands in den einzelnen Kantonen inkl. nationale Entwicklung (CH), 2013–2023</span>',
        'x': 0.01,
        'xanchor': 'left',
        'font': {'size': 14, 'color': IBCS_BLACK}
    },
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.99,
        'y': 1.12,
        'xanchor': 'right',
        'yanchor': 'top',
        'bgcolor': 'white',
        'bordercolor': IBCS_GRAY,
        'font': {'color': IBCS_BLACK}
    }],
    xaxis={
        'showgrid': False,
        'showline': True,
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticks': 'outside',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'title': {'text': 'Jahr', 'font': {'size': 10, 'color': IBCS_BLACK}},
        'tickmode': 'array',
        'tickvals': df_prozent['Jahr'],
        'range': [df_prozent['Jahr'].min() - 1, df_prozent['Jahr'].max() + 3]
    },
    yaxis={
        'showgrid': False,  # Grid einschalten für bessere Lesbarkeit
        'gridcolor': IBCS_LIGHT_GRAY,
        'gridwidth': 0.5,
        'showline': False,  # Y-Achsenlinie anzeigen
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticksuffix': '%',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'showticklabels': False,  # Y-Achsenbeschriftungen anzeigen
        'zeroline': False,  # Entfernt die Null-Linie, da wir diese bereits als Shape hinzufügen
        'range': [-3, 17]
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={'t': 100, 'b': 50, 'l': 60, 'r': 30},
    height=500,
    width=800,
    showlegend=False,  # Legende anzeigen
    annotations=[source_annotation, y_axis_title],
    shapes=[zero_line]
)

fig.show()

Insgesamt zeigt sich der höchste Zuwachs im Kanton Appenzell Innerrhoden mit einem Zuwachs von 15.4 % im Jahr 2020. Den stärksten Rückgang erlebte Basel Stadt im Jahr 2015, wobei der Fahrzeugbestand um 2.2 % sank.

In [ ]:
# Maximaler Zuwachs je Kanton + zugehöriges Jahr
maxima = df_prozent[kantone].max()
kanton_meiste_zuwachs = maxima.idxmax()
wert_meiste_zuwachs = maxima.max()
jahr_meiste_zuwachs = df_prozent.loc[df_prozent[kanton_meiste_zuwachs].idxmax(), 'Jahr']

# Minimaler Zuwachs je Kanton + zugehöriges Jahr
minima = df_prozent[kantone].min()
kanton_wenigste_zuwachs = minima.idxmin()
wert_wenigste_zuwachs = minima.min()
jahr_wenigste_zuwachs = df_prozent.loc[df_prozent[kanton_wenigste_zuwachs].idxmin(), 'Jahr']

# Ausgabe
print(f"Höchster Zuwachs: {kanton_meiste_zuwachs} mit {wert_meiste_zuwachs:.1f} % im Jahr {jahr_meiste_zuwachs}")
print(f"Stärkster Rückgang: {kanton_wenigste_zuwachs} mit {wert_wenigste_zuwachs:.1f} % im Jahr {jahr_wenigste_zuwachs}")


Höchster Zuwachs: LU mit 2.1 % im Jahr 2014
Stärkster Rückgang: LU mit -1.5 % im Jahr 2018


## Fahrzeugbestand nach Treibstoffart inkl. Entwicklung
In dieser Analyse wird der Fahrzeugbestand nach Treibstoffarten dargestellt. Dabei wurden neben den klassischen Treibstoffen wie Benzin und Diesel auch Fahrzeuge mit Elektromotor berücksichtigt, da diese zunehmend verbreitet sind. Andere Antriebsarten, wie zum Beispiel Wasserstoff und Gas, zeigen seit Jahren vergleichbare Werte und wurden daher in dieser Analyse nicht berücksichtigt.

In [ ]:
# Funktion zur Kategorisierung der Treibstoffe
def kategorisieren(treibstoff):
    elektro_hybrid = [
        'Benzin-elektrisch: Normal-Hybrid',
        'Benzin-elektrisch: Plug-in-Hybrid',
        'Diesel-elektrisch: Normal-Hybrid',
        'Diesel-elektrisch: Plug-in-Hybrid',
        'Elektrisch'
    ]
    benzin = [
        'Benzin'
    ]
    diesel = [
        'Diesel'
    ]

    # Nur relevante Antriebsarten berücksichtigen
    if treibstoff in elektro_hybrid:
        return 'Elektro/Hybrid'
    elif treibstoff in benzin:
        return 'Benzin'
    elif treibstoff in diesel:
        return 'Diesel'
    else:
        return None  # Alle anderen ausblenden

# Neue Spalte mit kategorisierten Antriebsarten
df_car_city['Antriebsart'] = df_car_city['Treibstoff'].apply(kategorisieren)

# Funktion zur Berechnung der Prognose für einen bestimmten Treibstoff
def forecast_by_treibstoff(df, treibstoff):
    df_filtered = df[df['Antriebsart'] == treibstoff]
    Jahr_spalten = [str(year) for year in range(2013, 2025)]  # Daten bis 2024 verwenden
    jahres_summe = df_filtered[Jahr_spalten].sum().fillna(0)

    if jahres_summe.sum() == 0:
        return None, None

    # X für die Jahre bis 2024 und y für die Fahrzeugzahlen bis 2024
    X = np.array(range(len(jahres_summe))).reshape(-1, 1)
    y = jahres_summe.values

    # Lineare Regression
    model = LinearRegression()
    model.fit(X, y)

    # Prognose ab 2025 bis 2027
    X_forecast = np.array(range(len(jahres_summe), len(jahres_summe) + 3)).reshape(-1, 1)  # Prognose nur bis 2027
    y_forecast = model.predict(X_forecast)

    # Jahre und Werte für Prognose
    Jahre = list(jahres_summe.index) + [str(year) for year in range(2025, 2028)]  # Prognose bis 2027
    Werte = list(jahres_summe.values) + list(y_forecast)

    return Jahre, Werte

# Erstellen des IBCS-Diagramms
def create_ibcs_plot(df):
    # Nur relevante Treibstoffe verwenden: Elektro/Hybrid, Benzin, Diesel
    antriebsarten = ['Elektro/Hybrid', 'Benzin', 'Diesel']
    fig = go.Figure()

    # Farben für historische und Prognose-Daten
    historical_color = 'black'  # Historische Daten in Schwarz
    forecast_color = 'lightgray'  # Prognose-Daten in Grau

    for antriebsart in antriebsarten:
        forecast_result = forecast_by_treibstoff(df, antriebsart)

        if forecast_result[0] is not None:
            Jahre, Werte = forecast_result

            # Historische Daten (bis 2024)
            fig.add_trace(go.Scatter(
                x=Jahre[:len(Jahre)-3],  # Bis 2024
                y=Werte[:len(Werte)-3],
                mode='lines+markers+text',
                name=f'{antriebsart} historisch',
                line=dict(color=historical_color),
                marker=dict(symbol='circle', size=6),
                text=[f'{round(val/1e6, 1)} M' for val in Werte[:len(Werte)-3]],  # Labels in Millionen
                textposition='top center',
                showlegend=False  # Keine Legende für historische Daten
            ))

            # Prognose-Daten (ab 2025 bis 2027)
            fig.add_trace(go.Scatter(
                x=Jahre[len(Jahre)-3:],  # Prognose für 2025 bis 2027
                y=Werte[-3:],  # Prognosewerte bis 2027
                mode='lines+markers+text',
                name=f'{antriebsart} prognose',
                line=dict(color=forecast_color, dash='dot'),
                marker=dict(symbol='circle-open', size=8),
                text=[f'{round(val/1e6, 1)} M' for val in Werte[-3:]],  # Labels in Millionen
                textposition='top center',
                showlegend=False  # Keine Legende für Prognose-Daten
            ))

            # Antriebsart am Ende der Linie beschriften
            # Verschieben Sie die Beschriftung der Antriebsart am Ende der Linie weiter nach links
            fig.add_trace(go.Scatter(
                x=[int(Jahre[-1]) - 0.2],  # Umwandlung von Jahre[-1] in int und Verschiebung nach links
                y=[Werte[-1]],  # Der letzte Wert der Prognose
                mode='text',
                name=f'{antriebsart} Antriebsart',
                text=[antriebsart],  # Antriebsart als Text
                textposition='middle center',  # Position der Beschriftung am Ende der Linie
                showlegend=False,  # Keine Legende für diese Beschriftung
                textfont=dict(size=11, color='black')
            ))

    # Layout
    fig.update_layout(
        title={
            'text': ('<b>Fahrzeugbestand nach Antriebsart</b><br>'
                     '<span style="font-size:14px;color:gray">'
                     'Fahrzeugbestand bis im Jahr 2024 sowie eine Prognose bis 2027 auf Basis einer linearer Regression (Datenbasis: 2013–2024)</span>'),
            'x': 0,
            'xanchor': 'left'  # Titel nach links verschieben
        },
        xaxis_title='Jahr',
        height=600,
        plot_bgcolor='white',
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            tickvals=[str(year) for year in range(2013, 2031)],  # Jahre bis 2028 einbeziehen
            ticktext=[str(year) for year in range(2013, 2028)] + ['']*4,  # Jahre bis 2024 anzeigen, 2025-2028 leer
            showticklabels=True  # Nur Jahre bis 2024 auf der x-Achse anzeigen
        ),
        yaxis=dict(showgrid=True, gridcolor='lightgray', zeroline=False, showticklabels=False),  # Y-Achse ohne Beschriftung
        showlegend=False  # Ganze Legende ausblenden
    )

    return fig

# Interaktives IBCS-Diagramm erzeugen
fig = create_ibcs_plot(df_car_city)
fig.show()


Die prozentuale Veränderung der Fahrzeugbestände in der Schweiz zwischen den Jahren 2013 und 2024 weist auf signifikante Trends in Bezug auf die verschiedenen Antriebsarten hin:

- Benzin: Der Bestand an Benzinfahrzeugen weist einen Rückgang von 14,42 % auf.
  
- Diesel: Die Dieselfahrzeuge hingegen haben um 9.99 % zugenommen.

- Elektro/Hybrid: Der Bestand von Elektro- und Hybridfahrzeugen weist einen signfikaten Anstieg von über 1563,80 % auf.

In [ ]:
# Funktion zur Berechnung der prozentualen Veränderung
def calculate_percentage_change(df, treibstoff):
    # Daten für den spezifischen Treibstoff herausfiltern
    df_filtered = df[df['Antriebsart'] == treibstoff]
    jahres_summe = df_filtered[[str(year) for year in range(2013, 2025)]].sum().fillna(0)

    if jahres_summe.sum() == 0:
        return None

    # Prozentsatzänderung berechnen (letzter Wert - erster Wert) / erster Wert * 100
    first_value = jahres_summe.iloc[0]
    last_value = jahres_summe.iloc[-1]
    percentage_change = ((last_value - first_value) / first_value) * 100

    return percentage_change

# Berechnung der prozentualen Veränderung für Diesel, Benzin und Elektro
benziner_change = calculate_percentage_change(df_car_city, 'Benzin')
diesel_change = calculate_percentage_change(df_car_city, 'Diesel')
elektro_change = calculate_percentage_change(df_car_city, 'Elektro/Hybrid')

# Ergebnisse anzeigen
print(f'Prozentuale Veränderung Benzin: {benziner_change:.2f}%')
print(f'Prozentuale Veränderung Diesel: {diesel_change:.2f}%')
print(f'Prozentuale Veränderung Elektro/Hybrid: {elektro_change:.2f}%')


Prozentuale Veränderung Benzin: -14.42%
Prozentuale Veränderung Diesel: 9.99%
Prozentuale Veränderung Elektro/Hybrid: 1563.80%


Entwicklung nach Treibstoffart und Kanton

In [ ]:
# Erweiterte Funktion zur Kategorisierung der Treibstoffe
def kategorisieren(treibstoff):
    elektro_hybrid = [
        'Benzin-elektrisch: Normal-Hybrid',
        'Benzin-elektrisch: Plug-in-Hybrid',
        'Diesel-elektrisch: Normal-Hybrid',
        'Diesel-elektrisch: Plug-in-Hybrid',
        'Elektrisch'
    ]
    benzin = ['Benzin']
    diesel = ['Diesel']

    if treibstoff in elektro_hybrid:
        return 'Elektro'
    elif treibstoff in benzin:
        return 'Benzin'
    elif treibstoff in diesel:
        return 'Diesel'
    else:
        return None

# Beispielhafte Daten bei fehlender df_car_city
try:
    df_car_city
except NameError:
    data = {
        'Kantonskürzel': ['ZH', 'BE', 'LU', 'ZH', 'BE', 'LU', 'ZH', 'BE', 'LU'],
        'Treibstoff': ['Benzin', 'Diesel', 'Elektrisch', 'Benzin-elektrisch: Plug-in-Hybrid',
                      'Diesel-elektrisch: Normal-Hybrid', 'Erdgas', 'Benzin', 'Diesel', 'Elektrisch']
    }
    jahre = [str(year) for year in range(2013, 2025)]
    for jahr in jahre:
        data[jahr] = [1000 + int(jahr) - 2013 * 100 + i * 50 for i in range(len(data['Kantonskürzel']))]
    df_car_city = pd.DataFrame(data)

# Kategorisierung und Filtern
df_car_city['Treibstoff_Kategorie'] = df_car_city['Treibstoff'].apply(kategorisieren)
df_filtered = df_car_city.dropna(subset=['Treibstoff_Kategorie'])
jahre = [str(year) for year in range(2013, 2025)]
df_kanton_treibstoff_jahr = df_filtered.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])[jahre].sum().reset_index()
df_long = pd.melt(df_kanton_treibstoff_jahr, id_vars=['Kantonskürzel', 'Treibstoff_Kategorie'], value_vars=jahre, var_name='Jahr', value_name='Anzahl')
df_long['Jahr'] = df_long['Jahr'].astype(int)
df_long = df_long.sort_values(['Kantonskürzel', 'Treibstoff_Kategorie', 'Jahr'])
df_long['pct_change'] = df_long.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])['Anzahl'].pct_change() * 100
df_prozent = df_long.dropna().copy()
df_prozent['Anzahl_Vorjahr'] = df_prozent.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])['Anzahl'].shift(1)
df_prozent['identifier'] = df_prozent['Kantonskürzel'] + '_' + df_prozent['Treibstoff_Kategorie']

# Farben
treibstoff_colors = {'Benzin': '#FF9933', 'Diesel': '#666666', 'Elektro': '#3366CC'}
ibcs_black = '#000000'
ibcs_gray = '#888888'
ibcs_light_gray = '#dddddd'
ibcs_green = '#00cc00'
ibcs_red = '#cc0000'

# Plot
fig = make_subplots(rows=1, cols=1)
unique_ids = df_prozent['identifier'].unique()
first_id = unique_ids[0] if len(unique_ids) > 0 else None

for identifier in unique_ids:
    subset = df_prozent[df_prozent['identifier'] == identifier]
    kanton, treibstoff = identifier.split('_')
    line_color = treibstoff_colors.get(treibstoff, ibcs_gray)
    marker_colors = [ibcs_green if val >= 0 else ibcs_red for val in subset['pct_change']]
    hover_texts = [
        f"{jahr}: {pct:.1f}% ({anz/1_000_000:.2f} Mio, Vorjahr: {vorjahr/1_000_000:.2f} Mio)"
        for jahr, pct, anz, vorjahr in zip(subset['Jahr'], subset['pct_change'], subset['Anzahl'], subset['Anzahl_Vorjahr'])
    ]

    fig.add_trace(go.Scatter(
        x=subset['Jahr'],
        y=subset['pct_change'],
        mode='lines+markers+text',
        name='',
        line=dict(color=line_color, width=1.5),
        marker=dict(color=marker_colors, size=6),
        text=[f"{a/1_000_000:.2f}M" for a in subset['Anzahl']],
        textposition='top center',
        hoverinfo='text',
        hovertext=hover_texts,
        visible=(identifier == first_id)
    ))

# Dropdown Buttons
buttons = []
kantone = sorted(set([identifier.split('_')[0] for identifier in unique_ids]))
for kanton in kantone:
    ids = [id for id in unique_ids if id.startswith(kanton + '_')]
    visibility = [id in ids for id in unique_ids]
    buttons.append(dict(
        label=kanton,
        method='update',
        args=[{'visible': visibility}]
    ))

# Layout aktualisieren
y_min = max(-30, df_prozent['pct_change'].min() * 1.2)
y_max = min(30, df_prozent['pct_change'].max() * 1.2)
fig.update_layout(
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.99,
        'y': 1.1,
        'xanchor': 'right',
        'yanchor': 'top',
        'bgcolor': 'white',
        'bordercolor': ibcs_gray,
        'font': {'color': ibcs_black}
    }],
    xaxis=dict(
        title='Jahr',
        showgrid=False,
        linecolor=ibcs_gray,
        ticks='outside',
        tickmode='array',
        tickvals=list(range(2014, 2025)),
        range=[2013.5, 2024.5]
    ),
    yaxis=dict(
        title='Veränderung in %',
        showgrid=True,
        gridcolor=ibcs_light_gray,
        linecolor=ibcs_gray,
        zeroline=False,
        range=[-15, 100]
    ),
    shapes=[dict(
        type='line',
        xref='paper', yref='y',
        x0=0, x1=1,
        y0=0, y1=0,
        line=dict(color=ibcs_black, width=1)
    )],
    annotations=[
        dict(
            x=0.01, y=-0.15,
            xref='paper', yref='paper',
            text='Quelle: Veränderung registrierter Fahrzeuge nach Kanton & Treibstoff, 2013–2024',
            showarrow=False,
            font=dict(size=8, color=ibcs_gray),
            align='left'
        ),
        dict(
            x=-0.08, y=0.5,
            xref='paper', yref='paper',
            text='Veränderung in %',
            showarrow=False,
            font=dict(size=10, color=ibcs_black),
            textangle=-90
        )
    ],
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={'t': 60, 'b': 80, 'l': 80, 'r': 60},
    height=600,
    width=900
)

fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Funktion zur Kategorisierung der Treibstoffe
def kategorisieren(treibstoff):
    elektro_hybrid = [
        'Benzin-elektrisch: Normal-Hybrid',
        'Benzin-elektrisch: Plug-in-Hybrid',
        'Diesel-elektrisch: Normal-Hybrid',
        'Diesel-elektrisch: Plug-in-Hybrid',
        'Elektrisch'
    ]
    benzin = ['Benzin']
    diesel = ['Diesel']

    if treibstoff in elektro_hybrid:
        return 'Elektro'
    elif treibstoff in benzin:
        return 'Benzin'
    elif treibstoff in diesel:
        return 'Diesel'
    else:
        return None  # Keine Zuordnung, daher None zurückgeben

# Angenommen df_car_city ist der ursprüngliche Datensatz
# Um den Code funktionsfähig zu machen, erzeugen wir Beispieldaten, falls df_car_city nicht existiert
try:
    df_car_city
except NameError:
    # Erstelle Beispieldaten für Test-Zwecke
    data = {
        'Kantonskürzel': ['ZH', 'BE', 'LU', 'ZH', 'BE', 'LU', 'ZH', 'BE', 'LU'],
        'Treibstoff': ['Benzin', 'Diesel', 'Elektrisch', 'Benzin-elektrisch: Plug-in-Hybrid',
                      'Diesel-elektrisch: Normal-Hybrid', 'Erdgas', 'Benzin', 'Diesel', 'Elektrisch']
    }

    # Füge Jahre hinzu
    jahre = [str(year) for year in range(2013, 2025)]
    for jahr in jahre:
        data[jahr] = [1000 + int(jahr) - 2013 * 100 + i * 50 for i in range(len(data['Kantonskürzel']))]

    df_car_city = pd.DataFrame(data)

# Kategorisiere die Treibstoffe
df_car_city['Treibstoff_Kategorie'] = df_car_city['Treibstoff'].apply(kategorisieren)

# Nur die gewünschten Kategorien behalten (Diesel, Benzin, Elektro), aber ohne 'None' Kategorien
df_filtered = df_car_city.dropna(subset=['Treibstoff_Kategorie'])

# Jahres-Spalten definieren
jahre = [str(year) for year in range(2013, 2025)]

# Gruppieren nach Kanton und Treibstoff_Kategorie (statt Treibstoff), dann Summieren der Fahrzeuge pro Jahr
df_kanton_treibstoff_jahr = df_filtered.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])[jahre].sum().reset_index()

# Ergebnis in Langformat umwandeln für einfachere Berechnung
df_long = pd.melt(
    df_kanton_treibstoff_jahr,
    id_vars=['Kantonskürzel', 'Treibstoff_Kategorie'],
    value_vars=jahre,
    var_name='Jahr',
    value_name='Anzahl'
)
df_long['Jahr'] = df_long['Jahr'].astype(int)

# Sortieren nach Kanton, Treibstoff_Kategorie und Jahr
df_long = df_long.sort_values(['Kantonskürzel', 'Treibstoff_Kategorie', 'Jahr'])

# Prozentuale Veränderung berechnen
df_long['pct_change'] = df_long.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])['Anzahl'].pct_change() * 100

# Entferne NaN-Werte (erstes Jahr für jeden Kanton und Treibstoff)
df_prozent = df_long.dropna().copy()

# Erstelle eindeutige Spaltenbezeichner für jede Kombination
df_prozent['identifier'] = df_prozent['Kantonskürzel'] + '_' + df_prozent['Treibstoff_Kategorie']

# IBCS Farbschema
IBCS_BLACK = '#000000'
IBCS_GRAY = '#888888'
IBCS_LIGHT_GRAY = '#dddddd'
IBCS_GREEN = '#00cc00'
IBCS_RED = '#cc0000'

# Farbzuordnung für die drei Treibstoffkategorien
TREIBSTOFF_COLORS = {
    'Benzin': '#FF9933',  # Orange
    'Diesel': '#666666',  # Dunkelgrau
    'Elektro': '#3366CC'  # Blau
}

# Funktion für Prozentformatierung
def format_percent(value):
    return f"{value:.1f}%"

# Plot erstellen
fig = make_subplots(rows=1, cols=1, subplot_titles=[""])

# Eindeutige Kombinationen von Kanton und Treibstoff identifizieren
unique_identifiers = df_prozent['identifier'].unique()

# Erste Kombination zur standardmässigen Anzeige
first_id = unique_identifiers[0] if len(unique_identifiers) > 0 else None

# Spuren für jede Kombination erstellen
for idx, identifier in enumerate(unique_identifiers):
    subset = df_prozent[df_prozent['identifier'] == identifier]
    kanton, treibstoff = identifier.split('_')

    text_values = [format_percent(val) for val in subset['pct_change']]

    # Benutze die Treibstoff-Farbe als Linienfarbe
    line_color = TREIBSTOFF_COLORS.get(treibstoff, IBCS_GRAY)

    # Marker-Farben basierend auf Werten (grün für positiv, rot für negativ)
    marker_colors = [IBCS_GREEN if val >= 0 else IBCS_RED for val in subset['pct_change']]

    trace = go.Scatter(
        x=subset['Jahr'],
        y=subset['pct_change'],
        mode='lines+markers',  # Linien und Marker verwenden
        name=f'{treibstoff} ({kanton})',  # Treibstoff + Kanton in der Legende anzeigen
        line=dict(color=line_color, width=2),
        marker=dict(color=marker_colors, size=8),
        text=text_values,
        textposition='top center',
        textfont=dict(size=9, color=IBCS_BLACK),
        hoverinfo='x+y+name',
        hovertemplate='%{x}: %{y:.1f}%<extra>%{name}</extra>',
        visible=(identifier == first_id)  # Nur erste Kombination standardmässig anzeigen
    )
    fig.add_trace(trace)

# Dropdown-Menü erstellen
buttons = []
kantone = sorted(set([identifier.split('_')[0] for identifier in unique_identifiers]))

for kanton in kantone:
    # Finde alle Identifikatoren, die zu diesem Kanton gehören
    kanton_identifiers = [id for id in unique_identifiers if id.startswith(kanton + '_')]

    # Erstelle die Visibility-Liste
    visible_list = [id in kanton_identifiers for id in unique_identifiers]

    buttons.append(dict(
        label=kanton,
        method='update',
        args=[{
            'visible': visible_list
        }, {
            'title': {
                'text': f'<b>Jährliche Veränderung des Fahrzeugbestands nach Treibstoff in {kanton}</b><br>'
                        f'<span style="font-size:10px;color:#888888">Prozentuale Veränderung des Fahrzeugbestands nach Kanton und Treibstoffart, 2013–2024</span>',
                'x': 0.01,
                'xanchor': 'left'
            }
        }]
    ))

# Y-Achsenbereich bestimmen, falls Daten vorhanden sind
if not df_prozent.empty:
    y_min = max(-30, df_prozent['pct_change'].min() * 1.2)
    y_max = min(30, df_prozent['pct_change'].max() * 1.2)
else:
    y_min, y_max = -10, 10  # Standardwerte, falls keine Daten vorhanden sind

# Shapes hinzufügen
shapes = [
    # Nulllinie
    dict(
        type='line',
        xref='paper',
        yref='y',
        x0=0,
        y0=0,
        x1=1,
        y1=0,
        line=dict(color=IBCS_BLACK, width=1)
    )
]

# Layout
first_kanton = kantone[0] if kantone else "Alle"
fig.update_layout(
    title={
        'text': f'<b>Jährliche Veränderung des Fahrzeugbestands nach Treibstoff in {first_kanton}</b><br>'
                f'<span style="font-size:10px;color:#888888">Prozentuale Veränderung des Fahrzeugbestands nach Kanton und Treibstoffart, 2013–2024</span>',
        'x': 0.01,
        'xanchor': 'left',
        'font': {'size': 14, 'color': IBCS_BLACK}
    },
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.99,
        'y': 1.12,
        'xanchor': 'right',
        'yanchor': 'top',
        'bgcolor': 'white',
        'bordercolor': IBCS_GRAY,
        'font': {'color': IBCS_BLACK}
    }],
    xaxis={
        'showgrid': False,
        'showline': True,
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticks': 'outside',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'title': {'text': 'Jahr', 'font': {'size': 10, 'color': IBCS_BLACK}},
        'tickmode': 'array',
        'tickvals': list(range(2014, 2025)) if not df_prozent.empty else list(range(2014, 2025)),
        'range': [2013.5, 2024.5] if not df_prozent.empty else [2013.5, 2024.5]
    },
    yaxis={
        'showgrid': True,  # Grid einschalten für bessere Lesbarkeit
        'gridcolor': IBCS_LIGHT_GRAY,
        'gridwidth': 0.5,
        'showline': True,  # Y-Achsenlinie anzeigen
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticksuffix': '%',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'showticklabels': True,  # Y-Achsenbeschriftungen anzeigen
        'zeroline': False,  # Entfernt die Null-Linie, da wir diese bereits als Shape hinzufügen
        'range': [y_min, y_max]
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={'t': 100, 'b': 60, 'l': 80, 'r': 80},
    shapes=shapes,
    legend={
        'title': {'text': 'Treibstoffkategorie', 'font': {'size': 10}},
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': -0.2,
        'xanchor': 'center',
        'x': 0.5
    },
    annotations=[
        # Quelle
        dict(
            x=0.01,
            y=-0.15,
            xref='paper',
            yref='paper',
            text='Quelle: Prozentuale Veränderung registrierter Fahrzeuge nach Kanton und Treibstoff, 2013–2024',
            showarrow=False,
            font=dict(size=8, color=IBCS_GRAY),
            align='left',
            xanchor='left'
        ),
        # Y-Achsenbeschriftung
        dict(
            x=-0.08,
            y=0.5,
            xref='paper',
            yref='paper',
            text='Veränderung in %',
            showarrow=False,
            font=dict(size=10, color=IBCS_BLACK),
            textangle=-90
        )
    ],
    height=600,
    width=900
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Funktion zur Kategorisierung der Treibstoffe
def kategorisieren(treibstoff):
    elektro_hybrid = [
        'Benzin-elektrisch: Normal-Hybrid',
        'Benzin-elektrisch: Plug-in-Hybrid',
        'Diesel-elektrisch: Normal-Hybrid',
        'Diesel-elektrisch: Plug-in-Hybrid',
        'Elektrisch'
    ]
    benzin = ['Benzin']
    diesel = ['Diesel']

    if treibstoff in elektro_hybrid:
        return 'Elektro'
    elif treibstoff in benzin:
        return 'Benzin'
    elif treibstoff in diesel:
        return 'Diesel'
    else:
        return None

# Beispieldaten erzeugen, falls df_car_city nicht existiert
try:
    df_car_city
except NameError:
    data = {
        'Kantonskürzel': ['ZH', 'BE', 'LU', 'ZH', 'BE', 'LU', 'ZH', 'BE', 'LU'],
        'Treibstoff': ['Benzin', 'Diesel', 'Elektrisch', 'Benzin-elektrisch: Plug-in-Hybrid',
                      'Diesel-elektrisch: Normal-Hybrid', 'Erdgas', 'Benzin', 'Diesel', 'Elektrisch']
    }
    jahre = [str(year) for year in range(2013, 2025)]
    for jahr in jahre:
        data[jahr] = [1000 + int(jahr) - 2013 * 100 + i * 50 for i in range(len(data['Kantonskürzel']))]
    df_car_city = pd.DataFrame(data)

# Kategorisierung
df_car_city['Treibstoff_Kategorie'] = df_car_city['Treibstoff'].apply(kategorisieren)
df_filtered = df_car_city.dropna(subset=['Treibstoff_Kategorie'])
jahre = [str(year) for year in range(2013, 2025)]

# Gruppieren und Umformen
df_kanton_treibstoff_jahr = df_filtered.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])[jahre].sum().reset_index()
df_long = pd.melt(
    df_kanton_treibstoff_jahr,
    id_vars=['Kantonskürzel', 'Treibstoff_Kategorie'],
    value_vars=jahre,
    var_name='Jahr',
    value_name='Anzahl'
)
df_long['Jahr'] = df_long['Jahr'].astype(int)
df_long = df_long.sort_values(['Kantonskürzel', 'Treibstoff_Kategorie', 'Jahr'])
df_long['pct_change'] = df_long.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])['Anzahl'].pct_change() * 100
df_prozent = df_long.dropna().copy()
df_prozent['identifier'] = df_prozent['Kantonskürzel'] + '_' + df_prozent['Treibstoff_Kategorie']

# Farben und Formatierungen
IBCS_BLACK = '#000000'
IBCS_GRAY = '#888888'
IBCS_LIGHT_GRAY = '#dddddd'

def format_percent(value):
    return f"{value:.1f}%"

# Plot
fig = make_subplots(rows=1, cols=1, subplot_titles=[""])
unique_identifiers = df_prozent['identifier'].unique()
first_id = unique_identifiers[0] if len(unique_identifiers) > 0 else None

# Spuren zeichnen
for idx, identifier in enumerate(unique_identifiers):
    subset = df_prozent[df_prozent['identifier'] == identifier]
    kanton, treibstoff = identifier.split('_')

    text_values = [format_percent(val) for val in subset['pct_change']]
    line_color = IBCS_BLACK
    marker_colors = [IBCS_BLACK for _ in subset['pct_change']]

    trace = go.Scatter(
        x=subset['Jahr'],
        y=subset['pct_change'],
        mode='lines+markers+text',
        name=f'{treibstoff} ({kanton})',
        line=dict(color=line_color, width=2),
        marker=dict(color=marker_colors, size=8),
        text=text_values,
        textposition='top center',
        textfont=dict(size=9, color=IBCS_BLACK),
        hoverinfo='x+y+name',
        hovertemplate='%{x}: %{y:.1f}%<extra>%{name}</extra>',
        visible=(identifier == first_id)
    )
    fig.add_trace(trace)

    # Beschriftung am letzten Punkt
    last_row = subset.iloc[-1]
    label_trace = go.Scatter(
        x=[last_row['Jahr']+0.5],
        y=[last_row['pct_change']],
        mode='text',
        text=[treibstoff],
        textposition='bottom right',
        textfont=dict(size=10, color=IBCS_BLACK),
        showlegend=False,
        hoverinfo='skip',
        visible=(identifier == first_id)
    )
    fig.add_trace(label_trace)

# Dropdown-Menü
buttons = []
kantone = sorted(set([identifier.split('_')[0] for identifier in unique_identifiers]))

for kanton in kantone:
    kanton_identifiers = [id for id in unique_identifiers if id.startswith(kanton + '_')]
    visibility = []
    for identifier in unique_identifiers:
        visibility.append(identifier in kanton_identifiers)
        visibility.append(identifier in kanton_identifiers)  # auch für Textspur
    buttons.append(dict(
        label=kanton,
        method='update',
        args=[{'visible': visibility},
              {'title': {
                  'text': f'<b>Jährliche Veränderung des Fahrzeugbestands nach Treibstoff in {kanton}</b><br>'
                          f'<span style="font-size:10px;color:#888888">Prozentuale Veränderung des Fahrzeugbestands nach Kanton und Treibstoffart, 2013–2024</span>',
                  'x': 0.01,
                  'xanchor': 'left'
              }}]
    ))

# Achsenbereich
if not df_prozent.empty:
    y_min = max(-30, df_prozent['pct_change'].min() * 1.2)
    y_max = min(30, df_prozent['pct_change'].max() * 1.2)
else:
    y_min, y_max = -10, 10

# Nulllinie
shapes = [dict(
    type='line',
    xref='paper',
    yref='y',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    line=dict(color=IBCS_BLACK, width=1)
)]

# Layout
first_kanton = kantone[0] if kantone else "Alle"
fig.update_layout(
    title={
        'text': f'<b>Jährliche Veränderung des Fahrzeugbestands nach Treibstoff in {first_kanton}</b><br>'
                f'<span style="font-size:10px;color:#888888">Prozentuale Veränderung des Fahrzeugbestands nach Kanton und Treibstoffart, 2013–2024</span>',
        'x': 0.01,
        'xanchor': 'left',
        'font': {'size': 14, 'color': IBCS_BLACK}
    },
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.99,
        'y': 1.12,
        'xanchor': 'right',
        'yanchor': 'top',
        'bgcolor': 'white',
        'bordercolor': IBCS_GRAY,
        'font': {'color': IBCS_BLACK}
    }],
    xaxis={
        'showgrid': False,
        'showline': True,
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticks': 'outside',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'title': {'text': 'Jahr', 'font': {'size': 10, 'color': IBCS_BLACK}},
        'tickmode': 'array',
        'tickvals': list(range(2014, 2025)),
        'range': [2013.5, 2027.5]
    },
    yaxis={
        'showgrid': False,
        'gridcolor': IBCS_LIGHT_GRAY,
        'gridwidth': 0.5,
        'showline': False,
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticksuffix': '%',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'showticklabels': False,
        'zeroline': False,
        'range': [-10, 77]
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={'t': 100, 'b': 60, 'l': 80, 'r': 80},
    shapes=shapes,
    legend={
        'title': {'text': 'Treibstoffkategorie', 'font': {'size': 10}},
        'orientation': 'h',
        'yanchor': 'bottom',
        'y': -0.2,
        'xanchor': 'right',
        'x': 0.5
    },
    annotations=[
        dict(
            x=0.01,
            y=-0.15,
            xref='paper',
            yref='paper',
            text='Quelle: Prozentuale Veränderung registrierter Fahrzeuge nach Kanton und Treibstoff, 2013–2024',
            showarrow=False,
            font=dict(size=8, color=IBCS_GRAY),
            align='left',
            xanchor='left'
        ),
        dict(
            x=-0.08,
            y=0.5,
            xref='paper',
            yref='paper',
            text='Veränderung in %',
            showarrow=False,
            font=dict(size=10, color=IBCS_BLACK),
            textangle=-90
        )
    ],
    height=600,
    width=900
)

fig.show()


In [ ]:
# Berechne die grösste prozentuale Veränderung pro Kanton und Treibstoff
max_pct_change = df_prozent.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])['pct_change'].max().reset_index()
min_pct_change = df_prozent.groupby(['Kantonskürzel', 'Treibstoff_Kategorie'])['pct_change'].min().reset_index()

# Finde die Top-Kantone mit den grössten positiven Änderungen
top_positive_changes = max_pct_change.sort_values(by='pct_change', ascending=False).head(5)

# Finde die Top-Kantone mit den grössten negativen Änderungen
top_negative_changes = min_pct_change.sort_values(by='pct_change', ascending=True).head(5)

# Ausgabe der Top 10 grössten positiven und negativen Veränderungen
top_positive_changes, top_negative_changes

(   Kantonskürzel Treibstoff_Kategorie  pct_change
 74            ZG              Elektro   75.247091
 65            UR              Elektro   69.659091
 71            VS              Elektro   67.477814
 29            GR              Elektro   66.995107
 32            JU              Elektro   65.510357,
    Kantonskürzel Treibstoff_Kategorie  pct_change
 61            TI               Diesel   -7.740563
 73            ZG               Diesel   -7.535558
 67            VD               Diesel   -7.119633
 22            GE               Diesel   -6.287176
 76            ZH               Diesel   -5.502278)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Funktion zur Kategorisierung der Treibstoffe
def kategorisieren(treibstoff):
    elektro_hybrid = [
        'Benzin-elektrisch: Normal-Hybrid',
        'Benzin-elektrisch: Plug-in-Hybrid',
        'Diesel-elektrisch: Normal-Hybrid',
        'Diesel-elektrisch: Plug-in-Hybrid',
        'Elektrisch'
    ]
    benzin = ['Benzin']
    diesel = ['Diesel']

    if treibstoff in elektro_hybrid:
        return 'Elektro'
    elif treibstoff in benzin:
        return 'Benzin'
    elif treibstoff in diesel:
        return 'Diesel'
    else:
        return None  # Keine Zuordnung, daher None zurückgeben

# Angenommen df_car_city ist der ursprüngliche Datensatz
# Kategorisiere die Treibstoffe
df_car_city['Treibstoff_Kategorie'] = df_car_city['Treibstoff'].apply(kategorisieren)

# Nur die gewünschten Kategorien behalten (Diesel, Benzin, Elektro), aber ohne 'None' Kategorien
df_filtered = df_car_city.dropna(subset=['Treibstoff_Kategorie'])
df_filtered = df_filtered[df_filtered['Treibstoff_Kategorie'].isin(['Diesel', 'Benzin', 'Elektro'])]

# Jahres-Spalten definieren
jahre = [str(year) for year in range(2013, 2025)]

# Gruppieren nach Kanton und Treibstoff, dann Summieren der Fahrzeuge pro Jahr
df_kanton_treibstoff_jahr = df_filtered.groupby(['Kantonskürzel', 'Treibstoff'])[jahre].sum().reset_index()

# Ergebnis in Langformat umwandeln für einfachere Berechnung
df_long = pd.melt(
    df_kanton_treibstoff_jahr,
    id_vars=['Kantonskürzel', 'Treibstoff'],
    value_vars=jahre,
    var_name='Jahr',
    value_name='Anzahl'
)
df_long['Jahr'] = df_long['Jahr'].astype(int)

# Sortieren nach Kanton, Treibstoff und Jahr
df_long = df_long.sort_values(['Kantonskürzel', 'Treibstoff', 'Jahr'])

# Prozentuale Veränderung berechnen
df_long['pct_change'] = df_long.groupby(['Kantonskürzel', 'Treibstoff'])['Anzahl'].pct_change() * 100

# Entferne NaN-Werte (erstes Jahr für jeden Kanton und Treibstoff)
df_prozent = df_long.dropna().copy()

# Erstelle eindeutige Spaltenbezeichner für jede Kombination
df_prozent['identifier'] = df_prozent['Kantonskürzel'] + '_' + df_prozent['Treibstoff']

# IBCS Farbschema
IBCS_BLACK = '#000000'
IBCS_GRAY = '#888888'
IBCS_LIGHT_GRAY = '#dddddd'
IBCS_GREEN = '#00cc00'
IBCS_RED = '#cc0000'

# Funktion für Prozentformatierung
def format_percent(value):
    return f"{value:.1f}%"

# Plot erstellen
fig = make_subplots(rows=1, cols=1, subplot_titles=[""])

# Eindeutige Kombinationen von Kanton und Treibstoff identifizieren
unique_identifiers = df_prozent['identifier'].unique()

# Erste Kombination zur standardmässigen Anzeige
first_id = unique_identifiers[0]

# Spuren für jede Kombination erstellen
for idx, identifier in enumerate(unique_identifiers):
    subset = df_prozent[df_prozent['identifier'] == identifier]
    kanton, treibstoff = identifier.split('_')

    text_values = [format_percent(val) for val in subset['pct_change']]
    colors = [IBCS_GREEN if val >= 0 else IBCS_RED for val in subset['pct_change']]

    trace = go.Scatter(
        x=subset['Jahr'],
        y=subset['pct_change'],
        mode='lines+markers',  # Linien und Marker verwenden
        name=f'{treibstoff} ({kanton})',  # Treibstoff + Kanton in der Legende anzeigen
        marker=dict(color=colors),
        text=text_values,
        textposition='top center',
        textfont=dict(size=9, color=IBCS_BLACK),
        hoverinfo='x+y+name',
        visible=(identifier == first_id)  # Nur erste Kombination standardmässig anzeigen
    )
    fig.add_trace(trace)

# Dropdown-Menü erstellen
buttons = []
kantone = sorted(set([identifier.split('_')[0] for identifier in unique_identifiers]))

for kanton in kantone:
    # Finde alle Identifikatoren, die zu diesem Kanton gehören
    kanton_identifiers = [id for id in unique_identifiers if id.startswith(kanton + '_')]

    # Erstelle die Visibility-Liste
    visible_list = [id in kanton_identifiers for id in unique_identifiers]

    buttons.append(dict(
        label=kanton,
        method='update',
        args=[{
            'visible': visible_list
        }, {
            'title': {
                'text': f'<b>Jährliche Veränderung des Fahrzeugbestands nach Treibstoff in {kanton}</b><br>'
                        f'<span style="font-size:10px;color:#888888">Prozentuale Veränderung des Fahrzeugbestands nach Kanton und Treibstoffart, 2013–2024</span>',
                'x': 0.01,
                'xanchor': 'left'
            }
        }]
    ))

# Y-Achsenbereich bestimmen
y_min = max(-3, df_prozent['pct_change'].min() * 1.2)
y_max = min(17, df_prozent['pct_change'].max() * 1.2)

# Annotationen
zero_line = dict(
    type='line',
    xref='paper',
    yref='y',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    line=dict(color=IBCS_BLACK, width=1)
)

source_annotation = dict(
    x=0.01,
    y=-0.15,
    xref='paper',
    yref='paper',
    text='Quelle: Prozentuale Veränderung registrierter Fahrzeuge nach Kanton und Treibstoff, 2013–2024',
    showarrow=False,
    font=dict(size=8, color=IBCS_GRAY),
    align='left',
    xanchor='left'
)

# Y-Achsenbeschriftung hinzufügen
y_axis_title = dict(
    x=-0.05,
    y=0.5,
    xref='paper',
    yref='paper',
    text='Veränderung in %',
    showarrow=False,
    font=dict(size=10, color=IBCS_BLACK),
    textangle=-90
)

# Layout
first_kanton = kantone[0]
fig.update_layout(
    title={
        'text': f'<b>Jährliche Veränderung des Fahrzeugbestands nach Treibstoff in {first_kanton}</b><br>'
                f'<span style="font-size:10px;color:#888888">Prozentuale Veränderung des Fahrzeugbestands nach Kanton und Treibstoffart, 2013–2024</span>',
        'x': 0.01,
        'xanchor': 'left',
        'font': {'size': 14, 'color': IBCS_BLACK}
    },
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 0.99,
        'y': 1.12,
        'xanchor': 'right',
        'yanchor': 'top',
        'bgcolor': 'white',
        'bordercolor': IBCS_GRAY,
        'font': {'color': IBCS_BLACK}
    }],
    xaxis={
        'showgrid': False,
        'showline': True,
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticks': 'outside',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'title': {'text': 'Jahr', 'font': {'size': 10, 'color': IBCS_BLACK}},
        'tickmode': 'array',
        'tickvals': sorted(df_prozent['Jahr'].unique()),
        'range': [df_prozent['Jahr'].min() - 0.5, df_prozent['Jahr'].max() + 0.5]
    },
    yaxis={
        'showgrid': True,  # Grid einschalten für bessere Lesbarkeit
        'gridcolor': IBCS_LIGHT_GRAY,
        'gridwidth': 0.5,
        'showline': True,  # Y-Achsenlinie anzeigen
        'linecolor': IBCS_GRAY,
        'linewidth': 1,
        'ticksuffix': '%',
        'tickfont': {'size': 9, 'color': IBCS_BLACK},
        'showticklabels': True,  # Y-Achsenbeschriftungen anzeigen
        'zeroline': False,  # Entfernt die Null-Linie, da wir diese bereits als Shape hinzufügen
        'range': [y_min, y_max]
    },
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin={'t': 100, 'b': 60, 'l': 80, 'r': 80}
)

fig.show()


In [ ]:
print(df_car_city.columns)


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress

# --- Nur reale Daten bis 2023 ---
df_real = df_merge_phev[df_merge_phev['Jahr'] <= 2023]

# --- Lineare Regression mit scipy ---
slope, intercept, r_value, p_value, std_err = linregress(
    df_real['Ladestationen_CH'],
    df_real['Elektro_PHEV_CH']
)

# --- Prognose: neue X-Werte (z. B. Ladestationen bis 2026) ---
lade_prognose = stations_pro_jahr[stations_pro_jahr['Jahr'] > 2023]
x_future = lade_prognose['Ladestationen_CH']
y_future = intercept + slope * x_future

# --- Plot ---
plt.figure(figsize=(10, 6))

# Originale Daten mit Regressionslinie
sns.regplot(
    data=df_real,
    x='Ladestationen_CH',
    y='Elektro_PHEV_CH',
    ci=None,
    scatter_kws={'color': 'darkorange'},
    line_kws={'color': 'black', 'label': 'Regression (2023)'}
)

# Prognosepunkte & Linie
plt.plot(x_future, y_future, 'o--', color='gray', label='Prognose (2024–2026)')

# Achsentitel & Info
plt.title(f'Elektro + Plug-in-Hybride vs. Ladestationen (CH)\nRegression bis 2023, Prognose bis 2026', fontsize=14)
plt.xlabel("Anzahl Ladestationen", fontsize=12)
plt.ylabel("Anzahl Fahrzeuge (Elektro + Plug-in)", fontsize=12)
plt.legend()
plt.tight_layout()
plt.grid(True)
plt.show()

# --- Info ausgeben ---
print(f"📊 Lineare Regression: y = {slope:.2f} * x + {intercept:.0f}")
print(f"📈 Pearson r = {r_value:.3f}, p = {p_value:.3g}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

# Lade die CSV-Datei
df_lade = pd.read_csv('/content/sample_data/ladestationen.csv')  # Pfad zur Datei anpassen

# --- 5. Ladestationen aggregieren ---
# Summiere die Spalten für jede Region (stations_*_count)
region_columns = [col for col in df_lade.columns if col.startswith('stations_') and col.endswith('count')]
df_lade['stations_CH_sum'] = df_lade[region_columns].sum(axis=1)

# Jetzt aggregieren wir die Gesamtanzahl der Ladestationen nach Jahr
stations_pro_jahr = df_lade.groupby('year')['stations_CH_sum'].sum().reset_index()
stations_pro_jahr.columns = ['Jahr', 'Ladestationen_CH']

# --- 1. Reine Elektrofahrzeuge in der Schweiz ---
elektro_ch = df_long[df_long['Treibstoff'] == 'Elektrisch']
elektro_pro_jahr_ch = elektro_ch.groupby('Jahr')['Fahrzeuganzahl'].sum().reset_index()
elektro_pro_jahr_ch.columns = ['Jahr', 'Elektro_CH']

# --- 2. Benzin-elektrisch: Plug-in-Hybrid ---
benzin_phev_ch = df_long[df_long['Treibstoff'] == 'Benzin-elektrisch: Plug-in-Hybrid']
benzin_phev_pro_jahr_ch = benzin_phev_ch.groupby('Jahr')['Fahrzeuganzahl'].sum().reset_index()
benzin_phev_pro_jahr_ch.columns = ['Jahr', 'Benzin_PHEV_CH']

# --- 3. Diesel-elektrisch: Plug-in-Hybrid ---
diesel_phev_ch = df_long[df_long['Treibstoff'] == 'Diesel-elektrisch: Plug-in-Hybrid']
diesel_phev_pro_jahr_ch = diesel_phev_ch.groupby('Jahr')['Fahrzeuganzahl'].sum().reset_index()
diesel_phev_pro_jahr_ch.columns = ['Jahr', 'Diesel_PHEV_CH']

# --- 4. Elektro + Plug-in-Hybride zusammenfassen ---
elektro_phev_ch = df_long[
    df_long['Treibstoff'].str.contains('Elektrisch|Plug-in-Hybrid', case=False, na=False)
]
elektro_phev_ch = elektro_phev_ch.groupby('Jahr')['Fahrzeuganzahl'].sum().reset_index()
elektro_phev_ch.columns = ['Jahr', 'Elektro_PHEV_CH']

# --- 6. Merge für reine Elektrofahrzeuge ---
df_merge_elektro = pd.merge(elektro_pro_jahr_ch, stations_pro_jahr, on='Jahr', how='inner')

# --- 7. Korrelation & Plot: Reine Elektrofahrzeuge ---
r_elektro, p_elektro = pearsonr(df_merge_elektro['Elektro_CH'], df_merge_elektro['Ladestationen_CH'])

print("Reine Elektrofahrzeuge vs. Ladestationen:")
print(df_merge_elektro)
print(f"📊 Pearson-Korrelation: r = {r_elektro:.3f}, p = {p_elektro:.3g}")

sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.regplot(data=df_merge_elektro, x='Ladestationen_CH', y='Elektro_CH', marker='o', ci=None)
plt.title(f'Elektrofahrzeuge vs. Ladestationen (CH)\nr = {r_elektro:.2f}, p = {p_elektro:.3g}')
plt.xlabel("Anzahl Ladestationen in der Schweiz")
plt.ylabel("Anzahl Elektrofahrzeuge in der Schweiz")
plt.tight_layout()
plt.show()

# --- 8. Merge für Benzin-elektrisch: Plug-in-Hybrid ---
df_merge_benzin_phev = pd.merge(benzin_phev_pro_jahr_ch, stations_pro_jahr, on='Jahr', how='inner')

# --- 9. Korrelation & Plot: Benzin-elektrisch: Plug-in-Hybrid ---
r_benzin_phev, p_benzin_phev = pearsonr(df_merge_benzin_phev['Benzin_PHEV_CH'], df_merge_benzin_phev['Ladestationen_CH'])

print("\nBenzin-elektrisch: Plug-in-Hybrid vs. Ladestationen:")
print(df_merge_benzin_phev)
print(f"📊 Pearson-Korrelation: r = {r_benzin_phev:.3f}, p = {p_benzin_phev:.3g}")

plt.figure(figsize=(8, 6))
sns.regplot(data=df_merge_benzin_phev, x='Ladestationen_CH', y='Benzin_PHEV_CH', marker='o', ci=None)
plt.title(f'Benzin-elektrisch: Plug-in-Hybrid vs. Ladestationen (CH)\nr = {r_benzin_phev:.2f}, p = {p_benzin_phev:.3g}')
plt.xlabel("Anzahl Ladestationen in der Schweiz")
plt.ylabel("Anzahl Benzin-elektrisch: Plug-in-Hybrid Fahrzeuge in der Schweiz")
plt.tight_layout()
plt.show()

# --- 10. Merge für Diesel-elektrisch: Plug-in-Hybrid ---
df_merge_diesel_phev = pd.merge(diesel_phev_pro_jahr_ch, stations_pro_jahr, on='Jahr', how='inner')

# --- 11. Korrelation & Plot: Diesel-elektrisch: Plug-in-Hybrid ---
r_diesel_phev, p_diesel_phev = pearsonr(df_merge_diesel_phev['Diesel_PHEV_CH'], df_merge_diesel_phev['Ladestationen_CH'])

print("\nDiesel-elektrisch: Plug-in-Hybrid vs. Ladestationen:")
print(df_merge_diesel_phev)
print(f"📊 Pearson-Korrelation: r = {r_diesel_phev:.3f}, p = {p_diesel_phev:.3g}")

plt.figure(figsize=(8, 6))
sns.regplot(data=df_merge_diesel_phev, x='Ladestationen_CH', y='Diesel_PHEV_CH', marker='o', ci=None)
plt.title(f'Diesel-elektrisch: Plug-in-Hybrid vs. Ladestationen (CH)\nr = {r_diesel_phev:.2f}, p = {p_diesel_phev:.3g}')
plt.xlabel("Anzahl Ladestationen in der Schweiz")
plt.ylabel("Anzahl Diesel-elektrisch: Plug-in-Hybrid Fahrzeuge in der Schweiz")
plt.tight_layout()
plt.show()

# --- 12. Merge für Elektro + Plug-in-Hybride ---
df_merge_phev = pd.merge(elektro_phev_ch, stations_pro_jahr, on='Jahr', how='inner')

# --- 13. Korrelation & Plot: Elektro + Plug-in ---
r_phev, p_phev = pearsonr(df_merge_phev['Elektro_PHEV_CH'], df_merge_phev['Ladestationen_CH'])

print("\nElektro + Plug-in-Hybride vs. Ladestationen:")
print(df_merge_phev)
print(f"📊 Pearson-Korrelation: r = {r_phev:.3f}, p = {p_phev:.3g}")

plt.figure(figsize=(8, 6))
sns.regplot(data=df_merge_phev, x='Ladestationen_CH', y='Elektro_PHEV_CH', marker='o', ci=None)
plt.title(f'Elektro + Plug-in-Hybride vs. Ladestationen (CH)\nr = {r_phev:.2f}, p = {p_phev:.3g}')
plt.xlabel("Anzahl Ladestationen in der Schweiz")
plt.ylabel("Anzahl Elektro + Plug-in-Hybrid Fahrzeuge in der Schweiz")
plt.tight_layout()
plt.show()


In [ ]:
# --- 1. Fahrzeuge (E + Plug-in-Hybrid) 2023 ---
jahre = [str(j) for j in range(2013, 2025)]
df_long = df_car_city.melt(
    id_vars=['Kantonskürzel', 'Treibstoff'],
    value_vars=jahre,
    var_name='Jahr',
    value_name='Fahrzeuganzahl'
)
df_long['Jahr'] = df_long['Jahr'].astype(int)

df_ephev = df_long[
    (df_long['Jahr'] == 2023) &
    (df_long['Treibstoff'].str.contains("Elektrisch|Plug-in-Hybrid", case=False, na=False))
]

fahrzeuge_2023 = df_ephev.groupby('Kantonskürzel')['Fahrzeuganzahl'].sum().reset_index()
fahrzeuge_2023.columns = ['Kanton', 'Fahrzeuge']

# --- 2. Strompreis 2023 – nur H1 und Standardprodukt ---
strom_2023 = df_strom[
    (df_strom['Periode'] == 2023) &
    (df_strom['Produkt'].str.lower().str.strip() == 'standardprodukt')
][['Kantonskürzel', 'Total exkl. MWST (Rp./kWH)']].copy()
strom_2023.columns = ['Kanton', 'Strompreis']

# --- 3. Ladestationen 2023 ---
station_cols = [col for col in df_lade.columns if col.startswith('stations_') and col.endswith('_count')]
df_lade['Jahr'] = df_lade['year']
stations_2023 = []

for col in station_cols:
    kanton = col.split('_')[1]
    temp = df_lade[df_lade['Jahr'] == 2023].groupby('Jahr')[col].sum().reset_index()
    temp['Kanton'] = kanton
    temp = temp.rename(columns={col: 'Ladestationen'})
    stations_2023.append(temp)

df_ladestationen_2023 = pd.concat(stations_2023)

# --- 4. Merge alles für 2023 ---
df_cluster_2023 = pd.merge(fahrzeuge_2023, strom_2023, on='Kanton')
df_cluster_2023 = pd.merge(df_cluster_2023, df_ladestationen_2023[['Kanton', 'Ladestationen']], on='Kanton')

# --- 5. KMeans-Clustering ---
scaler = StandardScaler()
X = df_cluster_2023[['Fahrzeuge', 'Ladestationen', 'Strompreis']]
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
df_cluster_2023['Cluster'] = kmeans.fit_predict(X_scaled)

# --- 6. Visualisierung ---
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_cluster_2023,
    x='Ladestationen',
    y='Fahrzeuge',
    hue='Cluster',
    palette='Set2',
    s=100
)

# Beschriftung mit Kantonskürzel
for _, row in df_cluster_2023.iterrows():
    plt.text(row['Ladestationen'] + 50, row['Fahrzeuge'] + 1000, row['Kanton'], fontsize=9)

plt.title("Clusteranalyse der Kantone (2023) – Standardprodukt")
plt.xlabel("Ladestationen (2023)")
plt.ylabel("E-Fahrzeuge + Plug-in-Hybride (2023)")
plt.legend(title="Cluster")
plt.tight_layout()
plt.show()

df_cluster_2023.to_csv('cluster_daten_2023.csv', index=False, encoding='utf-8')

In [ ]:
df_lade['Datum'] = pd.to_datetime(df_lade['year'].astype(str) + '-' + df_lade['month'].astype(str))
df_lade['Jahr'] = df_lade['Datum'].dt.year

# --- 2. Aggregiere Ladestationen pro Kanton & Jahr ---
station_cols = [col for col in df_lade.columns if col.startswith('stations_') and col.endswith('_count')]
stationen_pro_jahr = []

for col in station_cols:
    kanton = col.split('_')[1]
    temp = df_lade.groupby('Jahr')[col].sum().reset_index()
    temp['Kanton'] = kanton
    temp = temp.rename(columns={col: 'Ladestationen'})
    stationen_pro_jahr.append(temp)

df_ladestationen = pd.concat(stationen_pro_jahr)
df_ladestationen = df_ladestationen[df_ladestationen['Jahr'].between(2018, 2023)]

# --- 3. Strompreise vorbereiten ---
df_strom['Jahr'] = df_strom['Periode'].astype(int)
strom = df_strom[['Kantonskürzel', 'Jahr', 'Total exkl. MWST (Rp./kWH)']].copy()
strom.columns = ['Kanton', 'Jahr', 'Strompreis']

# --- 4. Fahrzeuge vorbereiten ---
jahre = [str(j) for j in range(2018, 2024)]
df_long = df_car_city.melt(
    id_vars=['Kantonskürzel', 'Treibstoff'],
    value_vars=jahre,
    var_name='Jahr',
    value_name='Fahrzeuganzahl'
)
df_long['Jahr'] = df_long['Jahr'].astype(int)

df_ephev = df_long[df_long['Treibstoff'].str.contains("Elektrisch|Plug-in-Hybrid", case=False, na=False)]
fahrzeuge = df_ephev.groupby(['Kantonskürzel', 'Jahr'])['Fahrzeuganzahl'].sum().reset_index()
fahrzeuge.columns = ['Kanton', 'Jahr', 'Fahrzeuge']

# --- 5. Zusammenführen ---
df_all = pd.merge(fahrzeuge, strom, on=['Kanton', 'Jahr'], how='inner')
df_all = pd.merge(df_all, df_ladestationen, on=['Kanton', 'Jahr'], how='inner')

# --- 6. Korrelationsmatrix ---
corr = df_all[['Fahrzeuge', 'Strompreis', 'Ladestationen']].corr()
plt.figure(figsize=(6, 4))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Korrelationsmatrix: Fahrzeuge – Strompreis – Ladestationen")
plt.tight_layout()
plt.show()


In [ ]:
df_cluster_2023['Kanton'] = df_cluster_2023['Kanton'].str.upper()

# --- 2. Koordinaten hinzufügen (für Bubble Map) ---
canton_coords = {
    "ZH": (47.3769, 8.5417), "BE": (46.9480, 7.4474), "LU": (47.0504, 8.3126), "UR": (46.8750, 8.6364),
    "SZ": (47.0243, 8.6573), "OW": (46.8750, 8.2533), "NW": (46.9490, 8.3750), "GL": (47.0400, 9.0660),
    "ZG": (47.1662, 8.5166), "FR": (46.8085, 7.1551), "SO": (47.2064, 7.5265), "BS": (47.5581, 7.5887),
    "BL": (47.4833, 7.7333), "SH": (47.6667, 8.6333), "AR": (47.3833, 9.4000), "AI": (47.3300, 9.4100),
    "SG": (47.4250, 9.3667), "GR": (46.6500, 9.6000), "AG": (47.3900, 8.0500), "TG": (47.5581, 9.0877),
    "TI": (46.3167, 8.8000), "VD": (46.7500, 6.6333), "VS": (46.2333, 7.8667), "NE": (46.9950, 6.8303),
    "GE": (46.2000, 6.1500), "JU": (47.3500, 7.1500)
}
df_cluster_2023['lat'] = df_cluster_2023['Kanton'].map(lambda x: canton_coords.get(x, (None, None))[0])
df_cluster_2023['lon'] = df_cluster_2023['Kanton'].map(lambda x: canton_coords.get(x, (None, None))[1])

# --- 3. Clustering ---
X = df_cluster_2023[['Fahrzeuge', 'Ladestationen', 'Strompreis']]
X_scaled = StandardScaler().fit_transform(X)

kmeans = KMeans(n_clusters=3, random_state=42, n_init='auto')
df_cluster_2023['Cluster'] = kmeans.fit_predict(X_scaled)

# --- 4. Mapping für GeoJSON-Namen (englisch, da wir gleich engl. GeoJSON nutzen) ---
kanton_map = {
    "ZH": "Zurich", "BE": "Bern", "LU": "Luzern", "UR": "Uri", "SZ": "Schwyz", "OW": "Obwalden", "NW": "Nidwalden",
    "GL": "Glarus", "ZG": "Zug", "FR": "Fribourg", "SO": "Solothurn", "BS": "Basel-Stadt", "BL": "Basel-Landschaft",
    "SH": "Schaffhausen", "AR": "Appenzell Ausserrhoden", "AI": "Appenzell Innerrhoden", "SG": "Sankt Gallen",
    "GR": "Graubunden", "AG": "Aargau", "TG": "Thurgau", "TI": "Ticino", "VD": "Vaud", "VS": "Valais",
    "NE": "Neuchatel", "GE": "Geneve", "JU": "Jura"
}
df_cluster_2023['Kantonsname'] = df_cluster_2023['Kanton'].map(kanton_map)
df_cluster_2023['Cluster'] = df_cluster_2023['Cluster'].astype(str)

# --- 6. Bubble Map (optional) ---
fig_bubble = px.scatter_mapbox(
    df_cluster_2023,
    lat='lat',
    lon='lon',
    size='Fahrzeuge',
    color='Cluster',
    color_discrete_sequence=px.colors.qualitative.Set2,
    hover_name='Kantonsname',
    hover_data=['Fahrzeuge', 'Ladestationen', 'Strompreis'],
    size_max=50,
    mapbox_style='carto-positron',
    zoom=6,
    center={"lat": 46.8, "lon": 8.3},
    title='🚗 E-Mobilitäts-Cluster pro Kanton (Bubble Map)'
)
fig_bubble.update_layout(height=600)
fig_bubble.show()
